In [1]:
import os
import sys
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import plotly
import plotly.graph_objs as go
import cufflinks as cf
cf.set_config_file(offline=True)

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
import os
import sys
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import plotly
import plotly.graph_objs as go
import cufflinks as cf
cf.set_config_file(offline=True)

In [3]:
base_path = os.path.abspath('../..')
data_path = os.path.join(base_path, 'data')
database_path = os.path.join(data_path, 'database')
data_path = os.path.join(base_path, 'data')

strategy_path = os.path.join(base_path, 'strategy')
check_path = os.path.join(strategy_path, 'check')
r_signal_path = os.path.join(check_path, 'signal')
sys.path.append(strategy_path)

In [4]:
from strategy import CommodityStrategy, EquityStrategy, IRStrategy, EmergingStrategy
from tester import Tester

In [5]:
from CPM import CPM
from CVA import CVA
from CVA2 import CVA2
from CSS import CSS
from CVO import CVO
from CCA3 import CCA3
from CVA3 import CVA3

In [6]:
# Comdty data
cindex_path = os.path.join(data_path, 'fut1return-com.CSV')
cindex2_path = os.path.join(data_path, 'BCOM.csv')

cindex = pd.read_csv(cindex_path, header=0, index_col=0, parse_dates=True)
cindex2 = pd.read_csv(cindex2_path, header=0, index_col=0, parse_dates=True)

CRet = cindex.pct_change(1).iloc[1:]
CRet2 = cindex2.pct_change(1).iloc[1:]
Cindex = (1. + CRet).cumprod()
Cindex = Cindex / Cindex.iloc[0]


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\jungyoon.choi\\Desktop\\jungyoon\\project\\risk_premia\\python\\data\\fut1return-com.CSV'

# 1. CPM

In [ ]:
cpm = CPM(strategy_name="CPM", asset_type="COMMODITY")
# cpm.load_index_and_return(from_db=True, save_file=True)

cpm.index = Cindex.copy()
cpm.ret = CRet.copy()
cpm.set_rebalance_period(ts_freq='week', cs_freq='week', rebalance_weekday=1)  # rebalance_day: monday = 0, sunday = 6 
cpm.calculate_signal(CS=0.35, minobs=52, longlen=52, shortlen=17)
cpm.set_portfolio_parameter(cs_strategy_type='vol')
cpm.make_portfolio()

In [ ]:
start_date = '2001-01-01'
end_date = '2018-05-09'

# get r results
cpm_comp_df = pd.read_csv('./past/cpm.csv', header=0, index_col=0, parse_dates=True)
cpm_lag_comp_df = pd.read_csv('./past/cpm_lag.csv', header=0, index_col=0, parse_dates=True)

In [ ]:
tester = Tester(cpm)
tester.set_period(start=start_date, end=end_date)
tester.run(save_file=False)

In [ ]:
to_plot = (1.+ cpm_lag_comp_df).loc[start_date:].cumprod()
to_plot['TS_python'] = tester.strategy.TS_result
to_plot['CS_python'] = tester.strategy.CS_result
to_plot.iplot()

# 2. CVA

In [ ]:
strategy_name = 'CVA'
cva = CVA(strategy_name="CVA", asset_type="COMMODITY")
# cva.load_index_and_return(from_db=False, save_file=False)
cva.index = Cindex.copy()
cva.ret = CRet.copy()

cva.set_rebalance_period(ts_freq='week', cs_freq='week', rebalance_weekday=1)  # rebalance_day: monday = 0, sunday = 6
cva.calculate_signal(CS=0.35, nopos=0.6, lookback_period=63)
##############  R SIGNAL 비교#############
TSRV = cva.TSRV
CSRV = cva.CSRV


R_TSRV =  pd.read_csv(os.path.join(r_signal_path, f'{strategy_name}_TSRV.csv'),
                      index_col = 'Unnamed: 0')

(R_TSRV- TSRV).abs().sum(1).iplot(title = f"{strategy_name}_TSRV")

R_CSRV =  pd.read_csv(os.path.join(r_signal_path, f'{strategy_name}_CSRV.csv'),
                      index_col = 'Unnamed: 0')

(R_CSRV- CSRV).abs().sum(1).iplot(title = f"{strategy_name}_CSRV")
###################################
cva.set_portfolio_parameter(cs_strategy_type='vol', min_vol=0.15)
cva.make_portfolio()

In [ ]:
start_date = '2001-01-01'
end_date = '2018-05-09'

# get r results
cva_comp_df = pd.read_csv('./past/cva.csv', header=0, index_col=0, parse_dates=True)
cva_lag_comp_df = pd.read_csv('./past/cva_lag.csv', header=0, index_col=0, parse_dates=True)

In [ ]:
tester = Tester(cva)
tester.set_period(start=start_date, end=end_date)
tester.run(save_file=False)

In [ ]:
to_plot = (1.+ cva_lag_comp_df).loc[start_date:].cumprod()
to_plot['TS_python'] = tester.strategy.TS_result
to_plot['CS_python'] = tester.strategy.CS_result
to_plot.iplot()

# 3. CVO

In [ ]:
cvo = CVO(strategy_name="CVO", asset_type="COMMODITY")
cvo.index = Cindex.copy()
cvo.ret = CRet.copy()

# cvo.load_index_and_return(from_db=False, save_file=False)
cvo.set_rebalance_period(ts_freq='week', cs_freq='week', rebalance_weekday=1)  # rebalance_day: monday = 0, sunday = 6
cvo.calculate_signal(CS=0.35, nopos=0.4, minobs=52)
cvo.set_portfolio_parameter(cs_strategy_type='vol')
cvo.make_portfolio()

In [ ]:
start_date = '2001-01-01'
end_date = '2018-05-09'

# get r results
cvo_comp_df = pd.read_csv('./past/cvo.csv', header=0, index_col=0, parse_dates=True)
cvo_lag_comp_df = pd.read_csv('./past/cvo_lag.csv', header=0, index_col=0, parse_dates=True)

In [ ]:
tester = Tester(cvo)
tester.set_period(start=start_date, end=end_date)
tester.run(save_file=False)

In [ ]:
to_plot = (1.+ cvo_lag_comp_df).loc[start_date:].cumprod()
to_plot['TS_python'] = tester.strategy.TS_result
to_plot['CS_python'] = tester.strategy.CS_result
to_plot.iplot()

# 4. CCA3

In [ ]:
cca3 = CCA3(strategy_name="CCA3", asset_type="COMMODITY")
cca3.index = Cindex.copy()
cca3.ret = CRet.copy()
# cca3.load_index_and_return(from_db=False, save_file=False)


carry_com = pd.read_csv(os.path.join(data_path, "carry-com.csv"), index_col=0, parse_dates=True)
carry_com.index.name = 'tdate'
carry_com.columns.name = 'ticker'

carry_com2 = pd.read_csv(os.path.join(data_path, "carry-com2.csv"), index_col=0, parse_dates=True)
carry_com2.index.name = 'tdate'
carry_com2.columns.name = 'ticker'

cca3.carry1 = carry_com
cca3.carry2 = carry_com2

# cca3.load_strategy_data(origin1='carry-com', origin2='carry-com2')
cca3.set_rebalance_period(ts_freq='month', cs_freq='month', rebalance_weekday=1)
cca3.calculate_signal(CS=0.35, nopos=0.4, minobs=60)
cca3.set_portfolio_parameter(cs_strategy_type='vol', min_vol=0.15)
cca3.make_portfolio()

In [ ]:
start_date = '2001-01-01'
end_date = '2018-05-09'

# get r results
cca3_comp_df = pd.read_csv('./past/cca3.csv', header=0, index_col=0, parse_dates=True)
cca3_lag_comp_df = pd.read_csv('./past/cca3_lag.csv', header=0, index_col=0, parse_dates=True)

In [ ]:
tester = Tester(cca3)
tester.set_period(start=start_date, end=end_date)
tester.run(save_file=False)

In [ ]:
to_plot = (1.+ cca3_lag_comp_df).loc[start_date:].cumprod()
to_plot['TS_python'] = tester.strategy.TS_result
to_plot['CS_python'] = tester.strategy.CS_result
to_plot.iplot()

# 5. CVA3

In [ ]:
cva3 = CVA3(strategy_name="CVA3", asset_type="COMMODITY")
cva3.index = Cindex.copy()
cva3.ret = CRet.copy()
# cva3.load_index_and_return(from_db=False, save_file=False)

cva3.set_rebalance_period(ts_freq='month', cs_freq='month')  # rebalance_day: monday = 0, sunday = 6

fx = pd.read_csv(os.path.join(data_path, "fx.csv"), index_col=0, parse_dates=True)
fx.index.name = 'tdate'
fx.columns.name = 'ticker'

fut1price = pd.read_csv(os.path.join(data_path, "fut1price-com.csv"), index_col=0, parse_dates=True)
fut1price.index.name = 'tdate'
fut1price.columns.name = 'ticker'

#cva3.load_strategy_data1(table='bloom', origin='fx')
#cva3.load_strategy_data2(table='past', origin='fut1price-com')
cva3.fx = fx
cva3.fut1price = fut1price

cva3.calculate_signal(CS=0.35, nopos=0.4, minobs=60, minobs1=12, SMA=1, LMA=12, Lwindow=54)
cva3.set_portfolio_parameter(cs_strategy_type='vol', min_vol=0.15)
cva3.make_portfolio()

In [ ]:
start_date = '2001-01-01'
end_date = '2018-05-09'

# get r results
cva3_comp_df = pd.read_csv('./past/cva3.csv', header=0, index_col=0, parse_dates=True)
cva3_lag_comp_df = pd.read_csv('./past/cva3_lag.csv', header=0, index_col=0, parse_dates=True)

In [ ]:
tester = Tester(cva3)
tester.set_period(start=start_date, end=end_date)
tester.run(save_file=False)

In [ ]:
to_plot = (1.+ cva3_lag_comp_df).loc[start_date:].cumprod()
to_plot['TS_python'] = tester.strategy.TS_result
to_plot['CS_python'] = tester.strategy.CS_result
to_plot.iplot()

# 6. CVA2

In [ ]:
cva2 = CVA2(strategy_name="CVA2", asset_type="COMMODITY")
# cva2.load_index_and_return(from_db=False, save_file=False)
cva2.index = Cindex.copy()
cva2.ret = CRet.copy()

cva2.set_rebalance_period(ts_freq='week', cs_freq='week')
# cva2.load_strategy_data(table='bloom', origin='commo pos')

commopos = pd.read_csv(os.path.join(data_path, "commo pos.csv"), index_col=0, parse_dates=True)
commopos.index.name = 'tdate'
commopos.columns.name = 'ticker'

cva2.commopos = commopos
cva2.calculate_signal(CS=0.35, nopos=0.4, minobs1=12)
cva2.set_portfolio_parameter(cs_strategy_type='vol')
cva2.make_portfolio()

In [ ]:
start_date = '2001-01-01'
end_date = '2018-05-09'

# get r results
cva2_comp_df = pd.read_csv('./past/cva2.csv', header=0, index_col=0, parse_dates=True)
cva2_lag_comp_df = pd.read_csv('./past/cva2_lag.csv', header=0, index_col=0, parse_dates=True)

In [ ]:
tester = Tester(cva2)
tester.set_period(start=start_date, end=end_date)
tester.run(save_file=False)

In [ ]:
to_plot = (1.+ cva2_lag_comp_df).loc[start_date:].cumprod()
to_plot['TS_python'] = tester.strategy.TS_result
to_plot['CS_python'] = tester.strategy.CS_result
to_plot.iplot()

# 7. CSS

In [ ]:
strategy_name = 'CSS'
css = CSS(strategy_name=strategy_name, asset_type="COMMODITY")
# css.load_index_and_return(from_db=False, save_file=False)
css.index = Cindex.copy()
css.ret = CRet.copy()

fx = pd.read_csv(os.path.join(data_path, "fx.csv"), index_col=0, parse_dates=True)
fx.index.name = 'tdate'
fx.columns.name = 'ticker'

carry_com = pd.read_csv(os.path.join(data_path, "carry-com.csv"), index_col=0, parse_dates=True)
carry_com.index.name = 'tdate'
carry_com.columns.name = 'ticker'

carry_com2 = pd.read_csv(os.path.join(data_path, "carry-com2.csv"), index_col=0, parse_dates=True)
carry_com2.index.name = 'tdate'
carry_com2.columns.name = 'ticker'

seasonal = pd.read_csv(os.path.join(data_path, "seasonal.csv"), index_col=0, parse_dates=True)
seasonal.index.name = 'tdate'
seasonal.columns.name = 'ticker'

css.fx = fx
css.carry_com = carry_com
css.seasonal = seasonal

# css.load_strategy_data1(table='bloom', origin1='fx', origin2='carry-com', origin3='carry-com2')
# css.load_strategy_data2(table='past', origin='seasonal')
css.set_rebalance_period(ts_freq='month', cs_freq='month', rebalance_weekday=1)  # rebalance_day: monday = 0, sunday = 6
css.calculate_signal(CS=0.35)
##############  R SIGNAL 비교#############
TSRV = css.TSRV
CSRV = css.CSRV


R_TSRV =  pd.read_csv(os.path.join(r_signal_path, f'{strategy_name}_TSRV.csv'),
                      index_col = 'Unnamed: 0')

(R_TSRV- TSRV).abs().sum(1).iplot(title = f"{strategy_name}_TSRV")

R_CSRV =  pd.read_csv(os.path.join(r_signal_path, f'{strategy_name}_CSRV.csv'),
                      index_col = 'Unnamed: 0')

(R_CSRV- CSRV).abs().sum(1).iplot(title = f"{strategy_name}_CSRV")
###################################

css.set_portfolio_parameter(cs_strategy_type='vol')


css.make_portfolio()

In [ ]:
start_date = '2001-01-01'
end_date = '2018-05-09'

# get r results
css_comp_df = pd.read_csv('./past/css.csv', header=0, index_col=0, parse_dates=True)
css_lag_comp_df = pd.read_csv('./past/css_lag.csv', header=0, index_col=0, parse_dates=True)

In [ ]:
tester = Tester(css)
tester.set_period(start=start_date, end=end_date)
tester.run(save_file=False)

In [ ]:
to_plot = (1.+ css_lag_comp_df).loc[start_date:].cumprod()
to_plot['TS_python'] = tester.strategy.TS_result
to_plot['CS_python'] = tester.strategy.CS_result
to_plot.iplot()